In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


1. IMPORT DATASET

In [39]:
#a/ Load the data set
df = pd.read_excel("E:\AIL_assignments\Assignment_1\Data_AIL.xlsx")
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [30]:
#b/ Select features
data = df[["start time","end time","No. Ingredients","viewer feeling of youtuber's style "]]

,start time,end time,No. Ingredients,viewer feeling of youtuber's style
2795,7:05,7:24,1,5


In [32]:
data.describe()

,No. Ingredients
count,3850.000000
mean,3.213506
std,2.812791
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,20.000000


In [33]:
data= data.dropna(axis= 0)
data.describe()

,No. Ingredients
count,3771.000000
mean,3.279236
std,2.804457
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,20.000000


In [23]:
# convert start time and end time to seconds
import re
def __check_value_time(hours=0, minutes=0, seconds=0):
    # print('Time out: 'hours, minutes, seconds)
    if hours > 24:
        print('Time error: ',hours, minutes, seconds)
        raise ValueError('Hour is out of range')

    if seconds > 60:
        print('Time error: ',hours, minutes, seconds)
        raise ValueError('Second is out of range')

    if minutes > 60:
        print('Time error: ',hours, minutes, seconds)
        raise ValueError('Minutes is out of range')

def process_time_str(time_str):
    time_data = re.findall('[0-9]?[0-9]', time_str)
    TIME_WITH_HOUR = 3
    TIME_WITH_MINUTES = 2
    TIME_WITH_SECONDS = 1
    MAX_SECONDS = 90060
    # print(time_data)
    if len(time_data) == TIME_WITH_HOUR:
        hours = int(time_data[0])
        minutes = int(time_data[1])
        seconds = int(time_data[2])

        __check_value_time(hours, seconds, minutes)

        return (hours*3600 + minutes*60 + seconds)
    if len(time_data) == TIME_WITH_MINUTES:
        minutes = int(time_data[0])
        seconds = int(time_data[1])

        __check_value_time(minutes=minutes, seconds=seconds)

        return (minutes*60 + seconds)
    if len(time_data) == TIME_WITH_SECONDS:
        seconds = int(time_data[0])

        __check_value_time(seconds=seconds)

        return seconds

In [26]:
index_data = list()
for str_time in data['start time']:
    if str_time[0].isdigit() and str_time[-1].isdigit():
        index_data.append(True)
    else:
        index_data.append(False)

data = data[index_data]

index_data = list()
for str_time in data['end time']:
    if str_time[0].isdigit() and str_time[-1].isdigit():
        index_data.append(True)
    else:
        index_data.append(False)
data = data[index_data]
df_time = list()

for start_time, end_time in zip(data['start time'], data['end time']):
    df_time.append((process_time_str(start_time), process_time_str(end_time)))

data[['start time', 'end time']] = df_time
data["viewer feeling of youtuber's style "] = data["viewer feeling of youtuber's style "].astype('int')


Time error:  31 0 44


ValueError: Hour is out of range

In [34]:
data.head()

,start time,end time,No. Ingredients,viewer feeling of youtuber's style
0,1:05,3:21,9,4
1,3:32,6:10,11,4
2,5:23,6:10,4,5
3,6:22,9:23,5,5
4,9:42,11:55,4,4


In [ ]:
#Scaling data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(data)

In [ ]:

#c/ Split it into two parts: 70% of which you will use to train your model and the remaining data will be used to test the model.
#
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.3,random_state=0)

2. Summarize and Visualize the dataset.

3. Build linear regression model

In [ ]:
#add collumn 1 in x
x_train = np.hstack((np.ones((x_train.shape[0],1)),x_train))
x_test = np.hstack((np.ones((x_test.shape[0],1)),x_test))

In [ ]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [ ]:

# linear regression using "mini-batch" gradient descent
# function to compute hypothesis / predictions
def hypothesis(X, theta):
    return np.dot(X, theta)
  
# function to compute gradient of error function w.r.t. theta
def gradient(X, y, theta):
    h = hypothesis(X, theta)
    grad = np.dot(X.transpose(), (h - y))
    return grad
  
# function to compute the error for current values of theta
def cost(X, y, theta,m):
    h = hypothesis(X, theta)
    J = np.dot((h - y).transpose(), (h - y))
    J /= (2 * m)
    return J[0][0]
  
# function to create a list containing mini-batches
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
  
    for i in range(n_minibatches):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches


  
# function to perform mini-batch gradient descent
def gradientDescent(X, y, learning_rate = 0.05, batch_size = 500):
    theta = np.zeros((X.shape[1], 1))
    theta_list = []
    cost_list = []
    max_iters = 5
    for itr in range(max_iters):
        mini_batches = create_mini_batches(X, y, batch_size)
        for mini_batch in mini_batches:
            X_mini, y_mini = mini_batch
            m = np.size(y_mini)
            theta = theta - (learning_rate/m) * gradient(X_mini, y_mini, theta)            
            cost_list.append(cost(X_mini, y_mini, theta,m))
            theta_list.append(theta)
    return  theta_list,cost_list

In [ ]:
theta_list,cost_list = gradientDescent(x_train,y_train)
i =cost_list.index(min(cost_list))
print("Min Cost: ",cost_list[i])
theta = theta_list[i]
print("Coeficient: ",theta)


4. Compare the results and make prediction

In [ ]:
# caculate r with python linear regession model  
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(x_train,y_train)
reg.score(x_test,y_test)

In [ ]:
# caculate r with the model above
from sklearn.metrics import r2_score 
y_predict = hypothesis(x_test,theta)
b = y_test
R_square = r2_score(y_test, y_predict) 
print('Coefficient of Determination', R_square) 

In [ ]:
#plot result
plt.figure(figsize=(15,10))
plt.scatter(y_test,y_predict)
plt.xlabel("Actual")
plt.ylabel("Predicted")


In [ ]:
#compare
result = pd.DataFrame({'Actual Value':y_test.reshape(y_test.shape[0],),'Predicted Value':y_predict.reshape(y_predict.shape[0],),"Difference": (y_test - y_predict).reshape(y_test.shape[0])})
result[0:20]